# Common Size Cash Flow

**Purpose**: The purpose of this notebook is to obtain the Cash Flow table from a Company filing (10-Q, 10-K) using the filing url. Edgar nicely provides just the table, which can be easily extracted using Pandas for further analysis. We want to convert the Cash Flows statement to a common size statement by dividing each year by the Net Income (Loss) amount. As a result, we can identify accounts that have the largest activity, relative to the Company's Net Income (Loss).

## Imports

In [1]:
from Helpers.CompanyFiling import Filing

## Set Up

Get the URL for the edgar filing. This is the "index-htm" page edgar directs you to when you click on a quarterly/annual report. We sould convert the url to redirect us to the folder structure of the filing package. Once inside, the folder has a file called the "FilingSummary.xml" which describes each item in the folder. We will scrape the filing summary to identify the file the Cash Flows statement is stored in. 

In [2]:
edgar_filing_url = "https://www.sec.gov/Archives/edgar/data/1617640/000161764021000012/0001617640-21-000012-index.html"
base_url, filing_summary_url = Filing.convert_filing_to_folder(edgar_filing_url=edgar_filing_url)
slugs = Filing.retrieve_face_report_slugs(filing_summary_url)
cash_flow_url = base_url + "/" + slugs["CF"]

## Clean

In order to analyze the Cash Flows statement, we first use pandas to read the table form the URL we discovered above. We clean the dataframe by renaming the first column header,  converting the multi-index columns to be a single index, and converting the data types to be numeric. Next, we remove rows where all numeric columns are all missing.

Lastly, we return the common size Cash Flows statement by dividng by the first row of the dataframe, which is where Net Income (Loss) is stored. We can confirm this by looking at the result. Net Income (Loss) in all years are equal to 1.0. 

In [3]:
cf = Filing.get_clean_table(cash_flow_url)
cf = cf.pipe(Filing.remove_rows_of_zeros)
cf = Filing.common_size_financial_statement(cf, divisor=cf.select_dtypes("number").iloc[0])

Unable to clean column - Captions.


## Visualize

We use pandas styler objects to add padding, add background colors, and format our numbers. As a result, we can easily identify where the Company is getting/spending its cash. 

In [4]:
cf.style\
    .background_gradient(cmap='RdBu', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .format({col: '{:,.2%}'.format for col in cf.select_dtypes("number").columns })\


,Captions,"Dec. 31, 2020","Dec. 31, 2019","Dec. 31, 2018"
1,Net loss,-100.00%,-100.00%,-100.00%
3,Depreciation and amortization,67.87%,28.64%,82.92%
4,Share-based compensation expense,121.86%,65.14%,124.38%
5,Amortization of right of use assets,15.01%,7.58%,0.00%
6,Amortization of contract cost assets,22.51%,11.57%,30.05%
7,"Amortization of discount and issuance costs on 2021 Notes, 2023 Notes, 2024 Notes, 2025 Notes, and 2026 Notes",63.17%,17.06%,22.25%
8,Gain on extinguishment of 2021 Notes,-0.89%,0.00%,0.00%
9,Impairment costs,47.37%,0.00%,65.91%
10,Deferred income taxes,-4.64%,-1.39%,-25.95%
11,Loss (gain) on disposal of property and equipment and other assets,-0.25%,2.37%,3.02%
